In [1]:
import argparse
import pickle
import numpy as np
import os
import tensorflow as tf
from tensorpack.utils import logger
from tensorpack import *
from tensorpack.tfutils.scope_utils import auto_reuse_variable_scope
# from tensorpack.tfutils import summary

from VAE.Model import ModelDesc, Trainer  # , RandomZData
from VAE.info_params import get_default_hparams
from VAE.load_data import *


# prefetch data
def get_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, train_next_shift, test_next_shift, train_target_one_hot, test_target_one_hot =\
    train_test_split(segment, next_segment, target_one_hot, hps)

    train_data = LoadData(train_segment, train_segment, train_target_one_hot, shuffle=True)
    test_data = LoadData(test_segment, test_segment, test_target_one_hot, shuffle=False)

    ds_train = ConcatData([train_data])
    ds_test = ConcatData([test_data])

    ds_train = BatchData(ds_train, batch_size=hps.M)
    ds_test = BatchData(ds_test, batch_size=hps.M)

    return ds_train, ds_test

def get_np_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, train_next_shift, test_next_shift, train_target_one_hot, test_target_one_hot =\
    train_test_split(segment, next_segment, target_one_hot, hps)

    return train_segment, test_segment, train_target_one_hot, test_target_one_hot

In [2]:

hps = get_default_hparams()

hps.T = 10

M = ModelDesc(hps)

In [3]:
train_segment, test_segment, train_target_one_hot, test_target_one_hot = get_np_data(hps)

Normalize: Z score


In [4]:
train_segment.shape

(15330, 10, 44)

In [5]:
test_segment.shape

(2303, 10, 44)

In [6]:
# data_point_0 = train_segment[0]
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize = (10,10))

plt.imshow(train_segment[1000], interpolation='nearest');
plt.colorbar()

In [7]:
28*44/128

9.625